# Reload XGBoost Model

## Import Packages

In [ ]:
# import packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import xgboost as xgb
from xgboost import Booster, DMatrix

## Load model and data

In [ ]:
xgb_loaded = Booster({'nthread': 4})  # init model
xgb_loaded.load_model('models/xgb/xgb.model')  # load data

In [ ]:
#import Data
df=pd.read_csv("./data/air_quality_final.csv")

## Preprocess data

In [ ]:
rseed = 42

X = df.drop('target', axis=1)
y = df['target']

#dummy-encode the location feature
location = pd.get_dummies(X['location'], prefix='location',drop_first=True)
# concatenate dummy-encoded locations feature to original dataframe
X = pd.concat([X, location],axis = 1)

#train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=X['location'], random_state=rseed)

#drop multiclass location column
X_train_loc = X_train.location
X_test_loc = X_test.location
X_train.drop('location', axis=1, inplace=True)
X_test.drop('location', axis=1, inplace=True)

## Predict on test set using re-loaded model

In [ ]:
# convert X_test into a xgb.DMatrix object and predict
y_pred_xgb_loaded = xgb_loaded.predict(DMatrix(X_test)) 

# evaluate model accuracy
rmse_xgb_loaded = mean_squared_error(y_test, y_pred_xgb_loaded, squared=False)
r2_xgb_loaded = r2_score(y_test, y_pred_xgb_loaded)

print(f'RMSE on testset: {round(rmse_xgb_loaded,2)}')
print(f'Coefficient of determination on testset: {round(r2_xgb_loaded,2)}')